In [9]:
import numpy as np
import os
import cv2
from PIL import Image
from scipy.ndimage import zoom
from scipy.signal import lfilter
from skimage.util import random_noise
from matplotlib import pyplot as plt

def elementaryCellularAutomata(rule, n, width=None, randfrac=None):
    # Validate inputs
    if not (0 <= rule <= 255):
        raise ValueError("Rule should be an integer between 0 and 255.")
    
    if not isinstance(n, int) or n <= 0:
        raise ValueError("N should be a positive integer.")
    
    if width is None:
        width = 2 * n - 1
    
    if isinstance(width, int):
        if width <= 0:
            raise ValueError("Width should be a positive integer.")
        patt = np.ones(width, dtype=int)
        patt[(width+1)//2 - 1] = 2
    else:
        if not all(map(lambda x: x in [0, 1], width)):
            raise ValueError("Width should be a list of 0s and 1s only.")
        patt = np.array(width) + 1
        width = len(patt)
    
    if randfrac is not None:
        if not (0 <= randfrac <= 1):
            raise ValueError("Randfrac should be a float between 0 and 1.")
        dorand = True
    else:
        dorand = False
    
    # Unpack rule
    rulearr = [(rule >> i) & 1 for i in range(8)]
    rulearr = np.array(rulearr) + 1
    
    # Initialize output pattern
    pattern = np.zeros((n, width), dtype=int)
    
    # Generate the pattern
    for i in range(n):
        pattern[i, :] = patt
        ind = 2**2 * np.roll(patt, 1) + 2**1 * patt + 2**0 * np.roll(patt, -1)
        ind = ind % 8  # Ensure indices are within 0-7
        patt = rulearr[ind]
        
        # Optional randomization
        if dorand:
            flip = np.random.rand(width) < randfrac
            patt[flip] = 3 - patt[flip]
    
    # Convert 1 and 2 to 0 and 1
    pattern -= 1
    
    return pattern

def caimgen(lp_length, imgsize):
    mask = elementaryCellularAutomata(np.random.randint(255), lp_length, lp_length, np.random.random())
    pix_value = np.random.rand(lp_length, lp_length) * 2 - 1
    imgseed = mask * pix_value
    
    imgraw = cv2.resize(imgseed, (imgsize * 2, imgsize * 2))  # Double the image size
    imgraw = imgraw[imgsize-1:2*imgsize-1, :]  # Crop the image
    return imgraw

def blurgen(img_gt, k):
    img_gtf = np.zeros_like(img_gt)
    
    # filter coefficients
    b = [0, k]
    a = [1, -(1-k)]
    
    for i in range(img_gt.shape[0]):
        img_gtf[i, :] = lfilter(b, a, img_gt[i, :])
    return img_gtf

k = [0.189*2, 0.189, 0.189/10, 0.189/100, 0.189/1000]

base_path = '/home/xavier/Documents/Tao-ImageSet/python_synthesized'

for fidx in range(1):
    folderpath = os.path.join(base_path, 'group_' + str(fidx))
    os.makedirs(folderpath, exist_ok=True)
    os.chdir(folderpath)
    
    lp_length = int(32 * 2**fidx)
    
    for j in k:
        subfname = str(j)
        os.makedirs(subfname, exist_ok=True)
        
        for i in range(1):
            img_temp = caimgen(lp_length, 1024)
            # Adding noise and blurring (assuming speckle noise and blur functions are defined)
            img_temp2 = random_noise(img_temp, mode='speckle') # img_temp + np.random.randn(*img_temp.shape)
            img_blur_temp = blurgen(img_temp2, j)
            
            img_gt = img_temp[:, 1024:]
            img_gtn = img_temp2[:, 1024:]
            img_blur = img_blur_temp[:, 1024:]
            fig, axs = plt.subplots(1, 3)
            cax1 = axs[0].imshow(img_gt)
            cax2 = axs[1].imshow(img_gtn)
            cax3 = axs[2].imshow(img_blur)
            # Add colorbars
            fig.colorbar(cax1, ax=axs[0])
            fig.colorbar(cax2, ax=axs[1])
            fig.colorbar(cax3, ax=axs[2])
            plt.show()
            # Image.fromarray(img_gt.astype(np.uint16)).save(f'{subfname}/gt_k={j}_{i}.tif')
            # Image.fromarray(img_gtn.astype(np.uint16)).save(f'{subfname}/gtn_k={j}_{i}.tif')
            # Image.fromarray(img_blur.astype(np.uint16)).save(f'{subfname}/k={j}_{i}.tif')
        
        # os.chdir(folderpath)


In [3]:
import numpy as np
from PIL import Image
import os

L = 1
edge = 1024 // L  # image horizontal pixel number, also number of comlume
ss = 1.5  # step size

N = int(1e7)  # number of steps
sN = 10  # number of random walkers
iN = 100  # number of images to be generated

def rand_img_gen(N, sN, edge, ss):
    k = 1
    seed = np.random.randint(0, edge, size=(sN, 2))
    img = np.zeros((edge, edge), dtype=np.uint16)
    for i in range(sN):
        img[seed[i, 0], seed[i, 1]] = 1
    traject = np.copy(seed)

    while k <= N:
        for i in range(sN):
            theta = np.random.rand() * 2 * np.pi
            dx = round(ss * np.cos(theta))
            dy = round(ss * np.sin(theta))
            x = traject[i, 0]
            y = traject[i, 1]

            x = correct_coordinate(x + dx, edge)
            y = correct_coordinate(y + dy, edge)

            img[x, y] += 1
            traject[i, 0] = x
            traject[i, 1] = y

        k += 1

    return img

def correct_coordinate(val, edge):
    if val <= 0:
        return abs(int(val))
    elif val >= edge:
        return 2*edge - int(val) - 2
    else:
        return int(val)

# Generate images
for i in range(iN):
    print(i)
    img_data = rand_img_gen(N, sN, edge, ss)
    img = Image.fromarray(img_data)
    img.save(os.path.join('images', f'RW_GT_I10_S1E7_{i}.tif'))